In [1]:
# TensorFlow and tf.keras
import tensorflow as tf

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import cv2

print(tf.__version__)

2.3.1


In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [3]:
tf.config.list_physical_devices('GPU')

[]

In [4]:
from tensorflow import keras
from tensorflow.keras import layers

In [5]:
#cuttle-environment-config mnist-pipeline task-id=fetch_data
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [6]:
#cuttle-environment-disable mnist-pipeline
!nvidia-smi

zsh:1: command not found: nvidia-smi


In [7]:
#cuttle-environment-config mnist-pipeline task-id=transform_data dependency=fetch_data

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [8]:
#cuttle-environment-config mnist-pipeline task-id=init_model dependency=transform_data

model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                1

In [9]:
#cuttle-environment-config mnist-pipeline task-id=train_model dependency=init_model,transform_data

batch_size = 128
epochs = 5

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/5
422/422 [==============================] - 17s 41ms/step - loss: 0.3623 - accuracy: 0.8880 - val_loss: 0.0838 - val_accuracy: 0.9770
Epoch 2/5
422/422 [==============================] - 17s 41ms/step - loss: 0.1149 - accuracy: 0.9649 - val_loss: 0.0559 - val_accuracy: 0.9852
Epoch 3/5
422/422 [==============================] - 17s 41ms/step - loss: 0.0868 - accuracy: 0.9729 - val_loss: 0.0475 - val_accuracy: 0.9868
Epoch 4/5
422/422 [==============================] - 17s 41ms/step - loss: 0.0715 - accuracy: 0.9780 - val_loss: 0.0435 - val_accuracy: 0.9880
Epoch 5/5
422/422 [==============================] - 18s 42ms/step - loss: 0.0617 - accuracy: 0.9807 - val_loss: 0.0383 - val_accuracy: 0.9885


In [15]:
#cuttle-environment-disable mnist-pipeline

a = open("./images/mnist3.png", "rb").read() #cuttle-environment-get-config mnist-api request.files.file

npimg = np.fromstring(a, np.uint8)

img = cv2.imdecode(npimg, cv2.IMREAD_UNCHANGED)
img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

x = np.invert(img)
x = cv2.resize(x,(28,28))
x = x.reshape(1,28,28,1)

digit = np.argmax(model.predict(x), axis=1)

c = str(digit)

print(c)

[3]


/Users/karishnu/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  


In [16]:
#cuttle-environment-config mnist-pipeline task-id=eval_model dependency=train_model,transform_data

score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.03932015225291252
Test accuracy: 0.9871000051498413


In [27]:
#cuttle-environment-disable mnist-pipeline

!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
